# Laser chambers (Jonathan D. Müller)

Loads the raw data of soil/branch chambers, applies corrections and calculates fluxes

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
# Initialization parameters

# Data input
project_path = './'

project_path_amb   = project_path + '02_preprocessed_data/ambient/'
project_path_laser = project_path + '02_preprocessed_data/laser computer/'
project_path_par   = project_path + '02_preprocessed_data/PAR/'
project_path_tc    = project_path + '02_preprocessed_data/thermocouples/'
project_path_flow  = project_path + '02_preprocessed_data/flow/'

# Additional data, logbook etc.
project_path_additional = project_path + '00_additional_data/'
cos_timing_fn       = 'COS_timing.xlsx'
chambers_sensors_fn = 'COS_chambers_sensor_logbook.xlsx'

# Output
project_path_output = project_path + '03_processed_data/'
output_fn = 'laser'

# List of months to process
# - If empty, all available data is processed, e.g. [  ]
# - Otherwise, specify using a string of year-month, e.g. ['2018-06'] or ['2017-03','2018-06']
month_list = ['2022-01']

# Main parameters
par_threshold    = 20     # Threshold by which to determine whether to keep PAR
averaging_period = 30     # How many [s] of each measurement are used to average the data. Currently given by the COS_timing file, but here can be shortened to <25s
keep_all_raw     = False   # If true, keep all data given by COS_timing when creating the raw output. Otherwise, keep the data of the averaging periody only
#dew_threshold    = 98     # Percentage of RH when dew starts to form, remove data when this is the case

# Choose how to find the ambient value for each measurement. There are 2 possibilities:
# - interpolated: This interpolates ambient measurements and adds them
# - next:         This simply finds the next ambient and adds it
#find_ambient     = 'next'
find_ambient     = 'interpolated'

# Leaf area parameters
proj_la_factor = np.pi # Factor used to convert total leaf area to projected leaf area

# leaf area index LAI
LAI_ctr = 1.5 # control
LAI_irr = 2.0 # irrigation

# Pump parameters: Values different from these will be deleted
min_abs_flow_branch = 3       # Minimum allowed pump flow for branch chambers
min_abs_flow_soil   = 0.5     # Minimum allowed pump flow for soil chambers
#max_delta_flow = 4     # Maximum difference between ambient and sample pumps
#max_stddev_flow = 0.5  # Maximum standard deviation within pump flows

# Calibration factor for gases
# - Factors by which the gases have to be corrected (Measured for our laser by Rafat Qubaja, Spring 2021)
fact_h2o = 1/1           # Calibration measured
fact_co2 = 396.5/378.52  # Calibration measured
fact_cos = 514/583.52    # Calibration measured
fact_co  = 1/1           # Not measured

# Day/Night PAR limit
# - Min. value of PAR when it's still day, using ambient PAR measurement
day_par_min = 10

## Load additionally needed data
- Leaf area logbook
- Chambers status (checks if the chamber was working or not)
- Timing information (when the laser was measuring what)
- PAR information (to which datalogger channel connected and if it's working)

In [ ]:
# Read additional data
#----------------------

# Read laser timing (lt) information, used to extract data when the chamber was closed
lt = pd.read_excel(project_path_additional + cos_timing_fn, sheet_name='timing')
lt.rename(columns={'date':'timestamp'}, inplace=True)
lt.drop(['comment'], axis=1, inplace=True)
lt['timestamp'] = pd.to_datetime( lt.timestamp, format='%Y-%m-%d', utc=True, errors='raise')
lt.dropna(subset=['timestamp'], inplace=True)
#display(lt)

# Read chamber status (cs)
cs = pd.read_excel(project_path_additional + chambers_sensors_fn, sheet_name='chambers')
cs.dropna(subset=['year'], inplace=True)
cs.drop(['comment'], axis=1, inplace=True)
# Convert dates
cs['timestamp'] = pd.to_datetime( cs.year.map(str) + '-' + cs.month.map(str) + '-' + cs.day.map(str), format='%Y-%m-%d', utc=True, errors='raise')
#cs['timestamp'] = pd.to_datetime( cs.year.map(str) + '-' + cs.month.map(str) + '-' + cs.day.map(str), format='%Y.0-%m.0-%d.0', utc=True, errors='raise')
# Filter out errors (too many days in a month)
cs = cs[cs.timestamp.notnull()]
#display(cs)

# Read PAR sensor installation and information
par_log = pd.read_excel(project_path_additional + chambers_sensors_fn, sheet_name='par')
par_log.dropna(subset=['date'], inplace=True)
par_log.rename(columns={'date':'timestamp'}, inplace=True)
par_log.drop(['comment'], axis=1, inplace=True)
#display(par_log)

# Log of flowmeter MUX channels in datalogger
fm_log = pd.read_excel(project_path_additional + chambers_sensors_fn, sheet_name='flow')
fm_log.dropna(subset=['date'], inplace=True)
fm_log.rename(columns={'date':'timestamp'}, inplace=True)
fm_log.drop(['comment'], axis=1, inplace=True)
#display(fm_log)

# Log of thermocouple MUX channels in datalogger
tc_log = pd.read_excel(project_path_additional + chambers_sensors_fn, sheet_name='tc')
tc_log.dropna(subset=['date'], inplace=True)
tc_log.rename(columns={'date':'timestamp'}, inplace=True)
tc_log.drop(['comment'], axis=1, inplace=True)
#display(tc_log)

# Log of surface area (branches and soil) in the chambers
sa_log = pd.read_excel(project_path_additional + chambers_sensors_fn, sheet_name='surf_area')
sa_log.dropna(subset=['date'], inplace=True)
sa_log.rename(columns={'date':'timestamp'}, inplace=True)
sa_log.drop(['comment'], axis=1, inplace=True)
#display(sa_log)

In [ ]:
# Debugging
#display(par_log)

# Functions

In [ ]:
# Physics: Unit converstions, fluxes, etc.
#-----------------------------------------

# Calculate the dewpoint temperature in C
def calculate_dewpointC(T_C, RH):
    dp = 243.04*(np.log(RH/100)+((17.625*T_C)/(243.04+T_C)))/(17.625-log(RH/100)-((17.625*T_C)/(243.04+T_C)))
    return(dp)

# Calculate saturation vapour pressure from pressure and temperature
# - 2 methods are available. Jones uses air pressure, Campbell & Norman do not
def calculate_es(T_C, P_Pa):
    # Jones p.348 (appendix 4)
    #es = (1.00072+(10**(-7)*P_Pa*(0.032+5.9*10**(-6)*T_C**2))) * (611.21*np.exp( (18.678-(T_C/234.5))*T_C/(257.14+T_C) ))

    # Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    # Campbell & Norman (1998)
    T_K = T_C + 273.15
    es = T_K**(-8.2) * np.exp(77.345 + 0.0057*T_K - 7235 * T_K**(-1))
    return(es)

# Calculates VPD from different environmental variables
def calculate_VPD(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]

    # From Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    e = h2o_mol_mol * P_Pa         # Water vapor partial pressure (Pa)
    es = calculate_es(T_C, P_Pa)   # Water vapor partial pressure at saturation (Pa)
    VPD = es - e                   # VPD (Pa)
    return(VPD)

# Converts water concentration [mmol mol] to RH [%]
def convert_mmol_RH(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]
    
    #es = calculate_es(T_C, P_Pa)
    #RH <- 0.263*P_Pa*((h2o_mmol_mol*18.02/28.97)/1000)*np.exp(17.67*(T_C)/(T_K-29.65))**(-1)
    #RH = 100 if (RH > 100) else RH
    #RH = np.nan if (RH < 5) else RH

    # From Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    e = h2o_mol_mol * P_Pa         # Water vapor partial pressure (Pa)
    es = calculate_es(T_C, P_Pa)   # Water vapor partial pressure at saturation (Pa)
    RH = e/es * 100                # RH (%)
    return(RH)

# Density of dry air
# - https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_rho_dry_air(T_C, h2o_mmol_mol, P_Pa):
    # Constants
    R_dry_air = 287.058     # [J kg-1 K-1] Specific gas const dry air
    T_K = T_C + 273.15
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol * P_Pa       # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    
    rho_dry_air = P_d / (R_dry_air * T_K) # Density of dry air (use for approximation)
    return(rho_dry_air)

# Density of moist air
def calculate_rho_moist_air(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]

    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol * P_Pa       # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mol_mol * M_h2o / v_a # Ambient water vapor mass density (kg m-3)

    # Moist air mass density (ρa, kg m-3) 
    rho_air = rho_d + rho_h2o

    return(rho_air)

# Dry air heat capacity at constant pressure
# cp_d in [J kg-1 K-1]
 # https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_cp_dry_air(T_C):
    cp = 1005 + ((T_C + 23.12)**2)/3364
    return(cp)

# Specific heat capacity of moist air at constant pressure
# cp_m in [J kg-1 K-1]
# https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_cp_moist_air(T_C, h2o_mmol_mol, P_Pa):
    # Unit conversions 
    T_K = T_C + 273.15           # Temperature in K
    h2o_mol_mol = h2o_mmol_mol * 10**(-3) # water in [mol mol-1]

    # RH
    RH = convert_mmol_RH(T_C, h2o_mmol_mol, P_Pa)

    # Water vapor heat capacity at constant pressure (cp_h2o, J kg-1 K-1)
    cp_h2o = 1859 + 0.13*RH + (0.193 + 5.6*10**(-3) * RH)*T_C + (10**(-3) + 5 * 10**(-5)*RH)*T_C**2

    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol * P_Pa       # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mol_mol * M_h2o / v_a # Ambient water vapor mass density (kg m-3)

    # Moist air mass density (ρa, kg m-3) 
    rho_air = rho_d + rho_h2o

    # Specific humidity (Q, kg kg-1) 
    Q = rho_h2o / rho_air

    # cp_moist
    cp = calculate_cp_dry_air(T_C) * (1-Q) + cp_h2o * Q
    return(cp)

# Calculates the flux of water
def calculate_h2o_flux(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, airflow_lpm, area_m2):
    # Unit conversions
    T_K = T_C + 273.15 # Temperature in K
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3)
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3)
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol_chamber * P_Pa # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (v_d, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (v_a, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3.s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol.s-1]
    
    # Note: We are measuring the outflow from the chamber, not the inflow (as opposed to the LI-6400)
    #       Therefore, we need to change eq. 1-2 (p. 1-7) to be ue = uo - sE
    #       This leads to eq. 1-3 changing to sE = uo*wo - (uo - sE)*we
    #       Finally, eq. 1-4 becomes E = uo*(wo - we) / (s*(1 - we))
    #       Where uo = airflow_mol_s
    #             wo = h2o_mol_mol_chamber (outgoing mol fraction)
    #             we = h2o_mol_mol_ambient (entering mol fraction)
    #             s  = area_m2 (leaf area)
      
    # LI-6400 manual, eq. 1-7
    h2o_flux = (airflow_mol_s * (h2o_mol_mol_chamber - h2o_mol_mol_ambient)) / (area_m2 * (1 - h2o_mol_mol_ambient))
    
    return(h2o_flux) # mol.m-2.s-1

# Inspired from LI-6400 manual: Uses the water flux because water changes the air density. If stomata are open and H2o is added, the gas is more 'diluted'
def calculate_gas_flux(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, gas_mol_mol_ambient, gas_mol_mol_chamber, airflow_lpm, area_m2):
    # Unit conversions
    T_K = T_C + 273.15 # Temperature in K
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3)
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3)
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    e = h2o_mol_mol_chamber * P_Pa # Water vapor partial pressure (Pa)
    P_d = P_Pa - e               # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3.s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol.s-1]
    
    # H2O transpiration flux [mol.m-2.s-1]
    h2o_flux_mol_m2_s = calculate_h2o_flux(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, airflow_lpm, area_m2)
    
    # Note: We are measuring the outflow from the chamber, not the inflow (as opposed to the LI-6400)
    #       Therefore, we need to change eq. 1-2 (p. 1-7) to be ue = uo - sE
    #       The LI-6400 uses assimilation, which is a positive flux. Previously, we used negative numbers due to CO2 uptake
    #       The sign should reflect the direction of the flux, i.e. emission is positive, and uptake negative, so assimilation should be negative
    #       The following calculations will therefore change eq. 1-11 (p. 1-9) to sa = uo*co - ue*ce
    #       This leads to eq. 1-12 (p. 1-9) changing to sa = uo*co - (uo - sE)*ce
    #       Finally, eq. 1-13 becomes a = uo*(co-ce) / s + E*ce
    #       Where uo = airflow_mol_s
    #             co = co2_mol_mol_chamber (outgoing mol fraction)
    #             ce = co2_mol_mol_ambient (entering mol fraction)
    #             s  = area_m2 (leaf area)
    #             E  = water flux (calculated above)
    
    # LI-6400 manual, eq. 1-13 (p. 1-10)
    gas_flux = (airflow_mol_s * (gas_mol_mol_chamber - gas_mol_mol_ambient)) / area_m2 + h2o_flux_mol_m2_s * gas_mol_mol_ambient
    
    return(gas_flux) # mol.m-2.s-1

# Conductance calculations
def calculate_cos_stomatal_conductance_ball_berry(T_C, h2o_mmol_mol, P_Pa, f_h2o_mmol_m2_s1, f_co2_umol_m2_s1, co2_umol_mol_ambient, PAR):
    # Note: T_C is the air temperature in C
    
    # Calculate some initial values
    VPD_Pa = calculate_VPD(T_C, h2o_mmol_mol, P_Pa)
    RH = convert_mmol_RH(T_C, h2o_mmol_mol, P_Pa)
    
    # Pressure in hPa
    P_kPa   = P_Pa / 1000
    VPD_kPa = VPD_Pa / 1000
    
    # Stomatal conductance
    g_s_cos = (f_h2o_mmol_m2_s1 * P_kPa * 10 / VPD_kPa) * 2 / 10000
    
    # Temporarily convert to df
    temp = pd.DataFrame({'rh': RH, 'par': PAR, 'f_co2': f_co2_umol_m2_s1, 'co2_a': co2_umol_mol_ambient, 'g_s': g_s_cos})
    
    # Ball-Berry correction
    temp['bbm'] = -temp['f_co2'] * temp['rh'] * 0.01 / temp['co2_a']
    temp.loc[(temp['rh'] > 70) & (temp['par'] > day_par_min), 'g_s'] = 17.207 * temp.loc[(temp['rh'] > 70) & (temp['par'] > day_par_min), 'bbm'] / 0.0487
    
    return(temp['g_s'])

def calculate_cos_total_conductance(f_cos_pmol_m2_s1, cos_pmol_mol_ambient):
    # Total conductance
    g_t_cos = - (f_cos_pmol_m2_s1 / cos_pmol_mol_ambient)
    return(g_t_cos)

def calculate_cos_internal_conductance(g_total, g_stomatal):
    # Total conductance
    g_i_cos = (g_total**(-1) - g_stomatal**(-1))**(-1)
    return(g_i_cos)

# Calculates leaf/soil relative uptake, i.e. LRU and SRU
def relative_uptake(f_cos_pmol_m2_s1, f_co2_umol_m2_s1, cos_pmol_mol_ambient, co2_umol_mol_ambient):
    ru = f_cos_pmol_m2_s1 / f_co2_umol_m2_s1 * co2_umol_mol_ambient / cos_pmol_mol_ambient
    return(ru)

def calculate_biochemical_conductance(T_leaf, LAI):
    # Preparations
    R  = 8.314463 # Ideal gas constant (J K-1 mol-1)
    E0 = 40
    ref_T_C = 20
    ref_T_K = ref_T_C + 273.15
    T_leaf_K = T_leaf + 273.15
    
    # Biochemical conductance
    g_ca = 0.8*0.055*LAI * np.exp((E0/R) * (1/ref_T_K - 1/T_leaf_K))

    return(g_ca)

def calculate_mesophyll_conductance(T_leaf, LAI):
    g_m = 0.188*LAI * np.exp(-0.5*(np.log((T_leaf/28.8)/0.61))**2)

    return(g_m)

In [ ]:
# Main functions
#---------------

# Delete all relevant files in a folder.
# - Used to remove 1min monthly files before running to prevent appending to existing files
def empty_dir(directory):
    files = glob.glob(directory + '*')
    for f in files:
        month_id = f[-10:-6] + '-' + f[-6:-4]
        if(month_id in month_list):
            print('Remove ' + f)
            os.remove(f)
        if(not month_list):
            print('Remove ' + f)
            os.remove(f)
    pass

# Read an input file
def read_data_file(input_fn):
    df = pd.read_csv(input_fn, na_values=['NAN'])
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors='raise')
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# This finds the file corresponding to a specific date and loads it
def find_and_load_file(fn_list, date_idx):
    data_fn = [i for i in fn_list if date_idx in i][0] if len([i for i in fn_list if date_idx in i]) > 0 else None
    if(data_fn is None):
        temp_df = read_data_file(fn_list[0])
        temp_df = temp_df.iloc[0:0]
    else:
        temp_df  = read_data_file(data_fn)
    return(temp_df)

# Assign block IDs for grouping and averaging in calculations
def create_block_id(temp):
    temp = temp.copy()
    blockID = 0
    blockIDs = []
    # Create an ID variable, used only here
    temp['id_var'] = temp['chamber'] + temp['status']
    x_old = temp['id_var'].tolist()[0]

    for x in temp['id_var'].tolist():
        # If same chamber, give same blockID
        if x == x_old:
            blockIDs.append(blockID)
            continue
        # If switch to new chamber, give new blockID
        else:
            blockID += 1
            blockIDs.append(blockID)
        x_old = x
    
    # Drop ID variable
    temp.drop('id_var', axis=1, inplace=True)
    
    # Add block ID
    temp.loc[:,'chamber_block'] = blockIDs
    
    return(temp)

# Calculate the standard deviation of means of means
# - provide a list of standard deviation columns of each initial mean
# - returns a list of standard deviations, by row
def std_means_of_means(sds):
        # Calculate a list of variances
        var = []
        for i in sds:
            var.append(i**2)
        # Calculate the mean
        var_mean = np.mean(var, axis=0)
        # Calculate the standard deviation
        sd_mean = np.sqrt(var_mean)
        return(sd_mean)

# Make means of all gases, because there are multiple measurements. Also add standard deviations
def fix_gas_units(temp):
    temp = temp.copy()
    # Convert units
    temp['OCS.1'] = temp['OCS.1'] * 1000
    temp['OCS.2'] = temp['OCS.2'] * 1000
    temp['CO2.1'] = temp['CO2.1'] / 1000
    temp['CO2.2'] = temp['CO2.2'] / 1000
    temp['CO2.3'] = temp['CO2.3'] / 1000
    temp['CO2.4'] = temp['CO2.4'] / 1000
    temp['H2O.1'] = temp['H2O.1'] / 10**6
    temp['CO.1'] = temp['CO.1'] * 1
    # drop bad columns
    temp.drop('CO2.3', axis=1, inplace=True)
    temp.drop('CO2.4', axis=1, inplace=True)
    # Rename columns
    temp.rename(columns={'OCS.1':'conc.cos.1.pmol_mol'}, inplace=True)
    temp.rename(columns={'OCS.2':'conc.cos.2.pmol_mol'}, inplace=True)
    temp.rename(columns={'CO2.1':'conc.co2.1.umol_mol'}, inplace=True)
    temp.rename(columns={'CO2.2':'conc.co2.2.umol_mol'}, inplace=True)
    temp.rename(columns={'H2O.1':'conc.h2o.mmol_mol'}, inplace=True)
    temp.rename(columns={'CO.1': 'conc.co.nmol_mol'}, inplace=True)
    # Apply calibration correction
    temp['conc.cos.1.pmol_mol'] = temp['conc.cos.1.pmol_mol'] * fact_cos
    temp['conc.cos.2.pmol_mol'] = temp['conc.cos.2.pmol_mol'] * fact_cos
    temp['conc.co2.1.umol_mol'] = temp['conc.co2.1.umol_mol'] * fact_co2
    temp['conc.co2.2.umol_mol'] = temp['conc.co2.2.umol_mol'] * fact_co2
    temp['conc.h2o.mmol_mol'] = temp['conc.h2o.mmol_mol'] * fact_h2o
    temp['conc.co.nmol_mol'] = temp['conc.co.nmol_mol'] * fact_co
    # Make mean of gases when multiple columns are available
    temp['conc.cos.mean.pmol_mol'] = temp[['conc.cos.1.pmol_mol','conc.cos.2.pmol_mol']].mean(axis=1)
    temp['conc.co2.mean.umol_mol'] = temp[['conc.co2.1.umol_mol','conc.co2.2.umol_mol']].mean(axis=1)
    # Add standard deviation of gases when multiple columns are available
    temp['conc.cos.mean_sd.pmol_mol'] = temp[['conc.cos.1.pmol_mol','conc.cos.2.pmol_mol']].std(axis=1)
    temp['conc.co2.mean_sd.umol_mol'] = temp[['conc.co2.1.umol_mol','conc.co2.2.umol_mol']].std(axis=1)
    return(temp)

# Identify the previous log date compared to the current date of the data
# This log date contains the relevant information to be read and applied
def find_log_entry(log_timestamps, this_date):
    previous_log_date = log_timestamps[0]
    log_entry_date = 0
    for i in log_timestamps:
        current_date = pd.to_datetime(str(this_date), format='%Y%m%d')
        # Find the entry that's in between the current date and the previous
        if((current_date > previous_log_date) and (current_date < i)):
            log_entry_date = previous_log_date
        # If the current date is larger than the last entry, keep the last entry
        if(current_date > log_timestamps[-1]):
            log_entry_date = i
        # If the entry is on a measurement day, make it 0 so that NAs are filled in
        if(current_date == i):
            log_entry_date = i
        previous_log_date = i
    return(log_entry_date)

# Read the log, and extract the relevant data
# Requires:
# - temp: dataframe with the data of the sensor (e.g. PAR, thermocouples, ...)
# - log_df: log-file dataframe
# - this_day: Current date
# - name: Name for the new columns (e.g., 'par')
# - unit: Unit of the measurement (e.g., 'umol_m2_s1' or 'c')
# - old_name: The start of the name of the column in the raw data (e.g., 'par_Avg(' or similar)
# - calibration: Factor by which the sensor needs to be multiplied to get the correct unit (standard 1, i.e. no change)
def get_relevant_data(temp, log_df, this_day, name, unit, old_name, calibration=1):
    raw_temp = temp.copy()
    
    # Identify the previous log date compared to the current date of the data
    # This log date contains the relevant information to be read and applied
    log_entry_date = find_log_entry(log_df['timestamp'].unique(), this_day)
    
    # Create empty PAR df that will be filled with PAR data
    # List all columns in the PAR log without timestamp to have a list of chamber names
    cols = log_df.columns.values.tolist()
    cols.remove('timestamp')
    # Copy all dates in the raw data that will be copied over
    out_df = raw_temp[['timestamp']].copy()
    # Create the empty df and fill it
    for col in cols:
        # Create column name
        col_name = name + '.' + col + '.' + unit
        # Initialise with NAs
        out_df[col_name] = np.nan
        # Identify MUX port and fill data in
        port = log_df.loc[(log_df['timestamp'] == log_entry_date), col].values[0]
        if(port > 0): # port 0 is basically bad data
            out_df[col_name] = raw_temp[old_name + str(int(port)) + ')']
        # If the sensor values need to be multiplied by a factor, e.g. to convert mV to umol.m-2.s-1
        out_df[col_name] = out_df[col_name] * calibration
    
    return(out_df)

def add_ambient(temp, amb_df):
    temp = temp.copy()
    
    # Drop unnecessary columns
    amb_df.drop('dayid', axis=1, inplace=True)
    
    # Merge PAR data
    merged = temp.merge(amb_df, on='timestamp', how='left')
    # Interpolate
    merged['temp.air.ambient.c'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['par.ambient.umol_m2_s1'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['conc.h2o.irga.ambient.mmol_mol'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['conc.co2.irga.ambient.umol_mol'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    
    return(merged)

# Adds sensor data and then interpolates it to fill the gaps
# - temp: Data df
# - sensor_df: Dataframe with sensor data
# - interpolation_limit: how many values are to be interpolated
def add_sensor_data(temp, sensor_df, interpolation_limit):
    temp = temp.copy()
    
    # List sensor columns only
    cols = sensor_df.columns.values.tolist()
    cols.remove('timestamp')
    
    # Merge sensor data
    merged = temp.merge(sensor_df, on='timestamp', how='left')
    # Interpolate sensor data
    for col in cols:
        merged[col].interpolate(method='linear', limit=interpolation_limit, limit_direction='both', limit_area='inside', inplace=True)
    
    return(merged)

def fix_PAR(temp):
    temp = temp.copy()
    
    # List PAR columns only
    cols = [col for col in temp.columns if 'par.' in col]
    
    # Night-time corrections, by cycling through chambers: Set to 0 during the night
    temp.loc[temp['par.ambient.umol_m2_s1'] < (par_threshold/2), 'par.ambient.umol_m2_s1'] = 0
    temp.loc[(temp['timestamp'].dt.hour < 5) | (temp['timestamp'].dt.hour > 21) , 'par.ambient.umol_m2_s1'] = 0 # Define the night by the maximum of June 21
    for col in cols:
        temp.loc[(temp[col] < par_threshold), col] = 0 # All PAR values should be 0 at night. Small values are errors
        temp.loc[(temp['par.ambient.umol_m2_s1'] == 0) & (temp[col] > par_threshold), col] = 0 # Remove large night-time values
        temp.loc[(temp['par.ambient.umol_m2_s1'] == 0) & (temp[col] > 0), col] = 0             # Apogee non-zero at night
        
    return(temp)

def make_counter_and_chamber(temp, this_day):
    temp = temp.copy()
    
    # Prepare the dataframe
    temp['type'] = np.nan
    temp['plot'] = np.nan
    temp['chamber'] = '' # Better to make it empty for the "create_block_id" function to be faster
    temp['status'] = ''  # Better to make it empty for the "create_block_id" function to be faster
    temp['counter'] = np.nan
    temp['chamber_block'] = np.nan
    
    # Extract the correct timing from the log file. If it's before the first available date, drop all data
    if(lt['timestamp'].unique()[0] >= pd.to_datetime(str(this_day), format='%Y%m%d', utc=True)): # Data from before the log file
        temp.drop(temp.index, inplace=True)
        return(temp)
    # Extract the correct timing from the log file
    relevant_date = lt.loc[lt['timestamp'] <= this_day,'timestamp'].unique()[-1]
    timing_dt = lt.loc[lt['timestamp'] == relevant_date]
    
    # In case the cycle is longer than 1h, adjust for it
    # For 1h long cycles, the times will be divided by 1 simply
    cycle_length = np.ceil(timing_dt['end_min'].max()/60)
    # Mark the chambers, and add chamber NA for data that needs to be removed
    temp['chamber'] = '' # remove data if it remains empty
    # Save the hour in the cycle
    temp['cycle_hour'] = temp['timestamp'].dt.strftime('%H').astype(float)
    temp['cycle_hour'] = temp['cycle_hour'].mod(cycle_length)
    # Create the minute relative to the hour in the cycle
    temp['chambertiming'] = temp['timestamp'].dt.strftime('%M%S').astype(float)
    temp['chambertiming'] = temp['chambertiming'] + temp['cycle_hour']*60*100
    # Walk through the log and save the data
    for index, row in timing_dt.iterrows():
        current_start = row['start_min']*100+row['start_sec']
        current_end   = row['end_min']*100+row['end_sec']
        # Name of current chamber
        if(row['status'] == 'amb'):
            current_ch = 'amb'
        else:
            current_ch = 'ch' + row['type'][0] + '_' + row['plot'] + str(int(row['num']))
        # Save the identifiers into the df
        temp.loc[(temp['chambertiming'] >= current_start) & \
                 (temp['chambertiming'] <= current_end), 'type'] = row['type']
        temp.loc[(temp['chambertiming'] >= current_start) & \
                 (temp['chambertiming'] <= current_end), 'plot'] = row['plot']
        temp.loc[(temp['chambertiming'] >= current_start) & \
                 (temp['chambertiming'] <= current_end), 'status'] = row['status']
        temp.loc[(temp['chambertiming'] >= current_start) & \
                 (temp['chambertiming'] <= current_end), 'chamber'] = current_ch
    temp.drop(['chambertiming', 'cycle_hour'], axis=1, inplace=True)
    
    # Add counter
    temp = create_block_id(temp)
    temp['counter'] = temp.groupby(['chamber_block']).cumcount()
    # Revert the counter to end at 0
    for name, group in temp.groupby(['chamber_block']):
        temp.loc[temp['chamber_block'] == name, 'counter'] = group['counter'] - group['timestamp'].count() + 1
    
    # Remove chamber number when not relevant
    temp.loc[~temp['status'].isin(['oc','cc','amb','']), 'chamber'] = ''
    # Make ambient plot into ambient
    temp.loc[temp['status'] == 'amb', 'plot'] = 'amb'
    
    # Chamber with NAs are a dummy that needs to be removed
    temp = temp.loc[temp['chamber'] != '']
    
    return(temp)

# Define day & night by PAR
def define_day_night(temp):
    temp = temp.copy()
    # Define nights as 0, days as 1
    temp['day_night'] = 0
    temp.loc[temp['par.ambient.umol_m2_s1'] > day_par_min, 'day_night'] = 1
    
    return(temp)

def add_current_variable(temp):
    temp = temp.copy()
    
    # Add a 'current' column for par temperature and flowrate, which has the reading for the current chamber
    temp['par.current.chamber.umol_m2_s1'] = np.nan
    temp['temp.leaf.current.chamber.c'] = np.nan
    temp['temp.soil.current.chamber.c'] = np.nan
    temp['temp.air.current.chamber.c'] = np.nan
    temp['pump.flow.current.chamber.slpm'] = np.nan
    # Loop through chambers and fill in the gaps
    chamber_list = temp['chamber'].dropna().unique().tolist()
    if('amb' in chamber_list): # Ambient 'chamber' should not be treated
        chamber_list.remove('amb')
    for i in chamber_list:
        if(('par.' + i + '.umol_m2_s1') in temp.columns.values):
            temp.loc[temp['chamber'] == i, 'par.current.chamber.umol_m2_s1'] = temp.loc[temp['chamber'] == i, 'par.' + i + '.umol_m2_s1']
        if(('temp.leaf.' + i + '.c') in temp.columns.values):
            temp.loc[temp['chamber'] == i, 'temp.leaf.current.chamber.c'] = temp.loc[temp['chamber'] == i, 'temp.leaf.' + i + '.c']
        if(('temp.soil.' + i + '.c') in temp.columns.values):
            temp.loc[temp['chamber'] == i, 'temp.soil.current.chamber.c'] = temp.loc[temp['chamber'] == i, 'temp.soil.' + i + '.c']
        if(('temp.air.' + i + '.c') in temp.columns.values):
            temp.loc[temp['chamber'] == i, 'temp.air.current.chamber.c'] = temp.loc[temp['chamber'] == i, 'temp.air.' + i + '.c']
        if(('pump.flow.' + i + '.slpm') in temp.columns.values):
            temp.loc[temp['chamber'] == i, 'pump.flow.current.chamber.slpm'] = temp.loc[temp['chamber'] == i, 'pump.flow.' + i + '.slpm']
    # When the chamber is NA, use ambient data
    temp.loc[temp['chamber'].isna(), 'par.current.chamber.umol_m2_s1'] = temp.loc[temp['chamber'].isna(), 'par.ambient.umol_m2_s1']
    temp.loc[temp['chamber'].isna(), 'temp.air.current.chamber.c'] = temp.loc[temp['chamber'].isna(), 'temp.air.ambient.c']
    return(temp)

def add_surface_area(temp, this_day, log_df):
    temp = temp.copy()
    
    # Identify the previous log date compared to the current date of the data
    # This log date contains the relevant information to be read and applied
    log_entry_date = find_log_entry(log_df['timestamp'].unique(), this_day)
    
    # List sensor columns only
    chamber_list = log_df.columns.values.tolist()
    chamber_list.remove('timestamp')
    
    # Add surface area
    temp['surface.area.m2'] = np.nan
    for ch in chamber_list:
        temp.loc[(temp['chamber'] == ch), 'surface.area.m2'] = log_df.loc[(log_df['timestamp'] == log_entry_date), ch].values[0]
    
    return(temp)

# Remove data from the bad chambers, based on the logbook
def add_log_flow(temp):
    temp = temp.copy()
    
    # Identify day
    day = temp['timestamp'].dt.normalize().unique()[0]
    
    # Keep only relevant days, make a list of bad chambers
    column_list = [cols for cols in cs.columns.tolist() if 'ch' in cols] # List only columns with 'ch' (i.e. chambers)
    cs_df = cs.loc[cs['timestamp'] == day, column_list]
    
    # Replace the pump flow if a constant was given in the logbook, or remove values if the pump wasn't working (according to the logbook)
    for chamber in cs_df.columns.tolist():
        if(cs_df[chamber].values[0] > 0):
            temp.loc[temp['chamber'] == chamber, 'pump.flow.current.chamber.slpm'] = cs_df[chamber].values[0]
        if(cs_df[chamber].values[0] == 0): # pump not working
            temp.loc[temp['chamber'] == chamber, 'pump.flow.current.chamber.slpm'] = np.nan
    
    # Add a column of unfiltered "current" flow data
    temp['pump.flow.unfiltered.current.chamber.slpm'] = temp['pump.flow.current.chamber.slpm']
    # Consider the pump as not working at a flow below the limit
    temp.loc[(temp['type'] == 'branch') & (temp['pump.flow.current.chamber.slpm'] <= min_abs_flow_branch), 'pump.flow.current.chamber.slpm'] = np.nan
    temp.loc[(temp['type'] == 'soil')   & (temp['pump.flow.current.chamber.slpm'] <= min_abs_flow_soil), 'pump.flow.current.chamber.slpm'] = np.nan
    
    return(temp)

# Remove data from the bad chambers, based on the logbook
def remove_bad_chambers(temp):
    temp = temp.copy()
    
    # Identify day
    day = temp['timestamp'].dt.normalize().unique()[0]
    
    # Keep only relevant days, make a list of bad chambers
    column_list = [cols for cols in cs.columns.tolist() if 'ch' in cols] # List only columns with 'ch' (i.e. chambers)
    cs_df = cs.loc[cs['timestamp'] == day, column_list]
    bad_chambers = cs_df.columns[(cs_df == 0).all()].tolist()
    
    # Show information
    print('    Non-working chambers (from logbook): ', bad_chambers)
    
    # Remove data when entire chamber was bad
    # COS (Carbonyl sulfide)
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.cos.1.pmol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.cos.2.pmol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.cos.mean.pmol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.cos.mean_sd.pmol_mol'] = np.nan
    # CO2 (Carbon dioxide)
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.co2.1.umol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.co2.2.umol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.co2.mean.umol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.co2.mean_sd.umol_mol'] = np.nan
    # H2O (Water)
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.h2o.mmol_mol'] = np.nan
    # CO (Carbon monoxide)
    temp.loc[temp['chamber'].isin(bad_chambers),'conc.co.nmol_mol'] = np.nan
    
    return(temp)

# Writes output files in full temporal resolution
def write_output_file(out_df, date_idx, out_dir):
    # Check if output folders exist. If not, create
    month_dir = str(date_idx)[0:4] + '-' + str(date_idx)[4:6]
    if(not os.path.exists(out_dir + month_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir + month_dir)
    # Create file name
    out_fn = out_dir + month_dir + '/' + output_fn + '_' + str(date_idx) + '.csv'
    print(str(len(out_df.dayid)), out_fn) # Shows final file size)
    # organise data for output
    temp_df = out_df
    # Before saving, remove the index
    temp_df.drop('dayid', axis=1, inplace=True)
    #temp_df.drop('chamber_block', axis=1, inplace=True)
    # Move timestamp column to the front
    col = temp_df.pop('counter')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('chamber_block')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('status')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('chamber')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('type')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('plot')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('timestamp')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    # Remove timezone information
    temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize(None)
    # Write data
    temp_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S') # Save file
    pass

# Removes all data that is not in the relevant timeframe
def remove_obsolete_data(temp, seconds):
    temp = temp.copy()
    # Remove data, if 'keep_all_raw' is 'True', keep all data
    if(keep_all_raw):
        pass
    else:
        temp = temp.loc[temp['counter'] > (-seconds)]
    return(temp)

# Calculate means and standard deviations of all data columns
def make_means(temp, seconds):
    temp = temp.copy()
    
    # Prepare timestamps: Convert to numbers so that averaging becomes possible
    temp['timestamp'] = temp['timestamp'].astype('int64')
    
    # Remove obsolete data if it wasn't removed previously
    if(~keep_all_raw):
        temp = temp.loc[temp['counter'] > (-seconds)]
    
    # Keep relevant identification columns
    id_cols = temp[['chamber_block','type','plot','chamber','status']].drop_duplicates()
    
    # Make standard deviations
    sd_df = temp.groupby(['chamber_block']).std()
    sd_df.reset_index(inplace=True)
    sd_df.drop('counter', axis=1, inplace=True)
    sd_df.drop('timestamp', axis=1, inplace=True)
    #sd_df.drop('chamber', axis=1, inplace=True)
    #sd_df.drop('plot', axis=1, inplace=True)
    sd_df.drop('surface.area.m2', axis=1, inplace=True)
    sd_df = sd_df.add_suffix('.sd')
    sd_df.rename(columns={'chamber_block.sd':'chamber_block'}, inplace=True)
    # Make means
    avg_df = temp.groupby(['chamber_block']).mean()
    avg_df.drop('counter', axis=1, inplace=True)
    avg_df.reset_index(inplace=True)
    # Convert averaged 'timestamps stored as numbers' back to (rounded) timestamps
    avg_df['timestamp'] = avg_df['timestamp'].astype('datetime64[ns]').dt.round('10s')
    
    # Merge means with standard deviations
    summarised = avg_df.merge(sd_df, on=['chamber_block'], how='left')
    # Remove weird indices
    summarised.reset_index(inplace=True)
    summarised.drop('index', axis=1, inplace=True)
    
    # Add identification columns (status and plot). We need to do this because text is not preserved when making averages
    summarised = summarised.merge(id_cols, on='chamber_block', how='left')
    # Move important columns to the front
    col = summarised.pop('status')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    col = summarised.pop('chamber')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    col = summarised.pop('plot')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    col = summarised.pop('type')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    col = summarised.pop('timestamp')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    
    # Fix: Calculate mean of means, and their standard deviations, of the gases with multiple columns (cos, co2)
    # (1) Remove previous standard deviations
    summarised.drop('conc.cos.mean_sd.pmol_mol', axis=1, inplace=True)
    summarised.drop('conc.co2.mean_sd.umol_mol', axis=1, inplace=True)
    summarised.drop('conc.cos.mean_sd.pmol_mol.sd', axis=1, inplace=True)
    summarised.drop('conc.co2.mean_sd.umol_mol.sd', axis=1, inplace=True)
    # (2) Calculate mean of means
    summarised['conc.cos.mean.pmol_mol'] = summarised[['conc.cos.1.pmol_mol','conc.cos.2.pmol_mol']].mean(axis=1)
    summarised['conc.co2.mean.umol_mol'] = summarised[['conc.co2.1.umol_mol','conc.co2.2.umol_mol']].mean(axis=1)
    # (3) Calculate standard deviation of 'means of means'
    summarised['conc.cos.mean.pmol_mol.sd'] = std_means_of_means([summarised['conc.cos.1.pmol_mol.sd'], summarised['conc.cos.2.pmol_mol.sd']])
    summarised['conc.co2.mean.umol_mol.sd'] = std_means_of_means([summarised['conc.co2.1.umol_mol.sd'], summarised['conc.co2.2.umol_mol.sd']])
    
    return(summarised)

# Interpolates ambient and open chamber readings into 1min readings. Then these are taken for the ambient reading
def find_corresponding_ambient_interpolated(temp):
    temp = temp.copy()
    
    # Create empty dataframe with a 1min frequency
    idx = pd.date_range(start=temp['timestamp'].tolist()[0],
                        end=temp['timestamp'].tolist()[-1],
                        freq='s')
    time_df = pd.DataFrame(idx, index=None, columns=['timestamp'])
    
    # Create a list of relevant columns
    column_list = [cols for cols in temp.columns.tolist() if 'conc' in cols] # List only columns with concentrations
    column_list = [cols for cols in column_list if not cols.endswith('.sd')] # Ignore standard deviation columns
    column_list = [cols for cols in column_list if not ('.irga.' in cols) and not ('.calibrated.' in cols)] # Ignore calibrated and IRGA columns
    
    # Extract ambient & interpolate
    ambient_df = time_df.merge(temp.loc[(temp['status'] == 'amb'),['timestamp'] + column_list], on='timestamp', how='left')
    col = ambient_df.pop('timestamp') # Interpolation doesn't work on timestamps, temporarily take it out
    ambient_df = ambient_df.interpolate(method='linear', limit=3600, limit_direction='both')
    ambient_df.insert(0, col.name, col, allow_duplicates=True) # And put the timestamp back
    # Rename columns
    ambient_df = ambient_df.add_suffix('.ambient')
    ambient_df.rename(columns={'timestamp.ambient':'timestamp'}, inplace=True)
    
    # Extract open chamber & interpolate, if it exists. Create an empty df if not
    oc_data_list = []
    chamber_list = temp['chamber'].unique().tolist()
    # Interpolate chamber by chamber
    for ch in chamber_list:
        if('oc' in temp['status'].unique()):
            oc_temp = time_df.merge(temp.loc[(temp['chamber'] == ch) & (temp['status'] == 'oc'),['timestamp'] + column_list], on='timestamp', how='left')
            col = oc_temp.pop('timestamp') # Interpolation doesn't work on timestamps, temporarily take it out
            oc_temp = oc_temp.interpolate(method='linear', limit=3600, limit_direction='both')
            oc_temp.insert(0, col.name, col, allow_duplicates=True) # And put the timestamp back
        else: # Add multiple columns to the df, but leave them empty
            oc_temp = time_df.reindex(columns=['timestamp']+column_list)
        # Add chamber label
        oc_temp['chamber'] = ch
        oc_data_list.append(oc_temp)
    oc_df = pd.concat(oc_data_list) # Now merge all
    # Rename columns
    oc_df = oc_df.add_suffix('.oc')
    oc_df.rename(columns={'timestamp.oc':'timestamp'}, inplace=True)
    oc_df.rename(columns={'chamber.oc':'chamber'}, inplace=True)
    
    # Merge with data. Note: Removes the ambient from the temp df before merging, so that no ambient-to-ambient flux is calculated
    merged = temp.loc[temp['chamber'] != 'amb'].merge(ambient_df, on='timestamp', how='left')
    merged = merged.merge(oc_df, on=['timestamp','chamber'], how='left')
    
    return(merged)

# Find the next ambient or open chamber after the current measurement, mark it, then append it
def find_corresponding_ambient_next(temp):
    temp = temp.copy()
    
    # Create a list of relevant columns
    column_list = [cols for cols in temp.columns.tolist() if 'conc' in cols] # List only columns with concentrations
    column_list = [cols for cols in column_list if not cols.endswith('.sd')] # Ignore standard deviation columns
    column_list = [cols for cols in column_list if not ('.irga.' in cols) and not ('.calibrated.' in cols)] # Ignore calibrated and IRGA columns
    
    # Extract ambient & prepare
    amb_df = temp.loc[(temp['status'] == 'amb'),['chamber_block'] + column_list].copy()
    amb_df = amb_df.add_suffix('.ambient') # Rename columns
    amb_df.rename(columns={'chamber_block.ambient':'chamber_block'}, inplace=True)
    # Extract open chamber & prepare
    oc_df = temp.loc[(temp['status'] == 'oc'),['chamber_block'] + column_list].copy()
    oc_df = oc_df.add_suffix('.oc') # Rename columns
    oc_df.rename(columns={'chamber_block.oc':'chamber_block'}, inplace=True)
    # Ambient column to hold the chamber block number
    amb_df['amb_block'] = np.nan
    oc_df['amb_block'] = np.nan
    
    # Search for the corresponding next ambient or open chamber, do it chamber by chamber
    chamber_list = temp['chamber'].unique().tolist()
    amb_list = [ ]
    for ch in chamber_list:
        temp_amb = amb_df.copy() # Ambient for each chamber is treated separately, and then concatenated (below)
        # Cycle through closed chambers
        cc_block_list = temp.loc[(temp['status'] == 'cc') & (temp['chamber'] == ch), 'chamber_block'].unique().tolist()
        # Make sure the open chamber is the same chamber
        oc_block_list = temp.loc[(temp['status'] == 'oc') & (temp['chamber'] == ch), 'chamber_block'].unique().tolist()
        # For ambient, we don't have to take the same chamber
        amb_block_list = temp.loc[(temp['status'] == 'amb'), 'chamber_block'].unique().tolist()
        
        for block in cc_block_list:
            # Find next block with amb or oc. Avoid ValueError if the list is empty
            if(len(oc_block_list) > 0):
                if(block < np.max(oc_block_list)):
                    oc_block =  np.min([x for x in oc_block_list if x > block])
            else:
                oc_block =  np.nan
            if(len(amb_block_list) > 0):
                if(block < np.max(amb_block_list)):
                    amb_block =  np.min([x for x in amb_block_list if x > block])
            else:
                amb_block =  np.nan
            # In the ambient and OC dfs, replace the chamber_block with the current block
            temp_amb.loc[temp_amb['chamber_block'] == amb_block, 'amb_block'] = block
            oc_df.loc[oc_df['chamber_block'] == oc_block, 'amb_block'] = block
        # If the current chamber is not ambient, add this to the list of ambient blocks that were found
        if(ch != 'amb'):
            amb_list.append(temp_amb.loc[~temp_amb['amb_block'].isna()])
    amb_df = pd.concat(amb_list)
    
    # Sometimes, the is another OC or ambient measured before a chamber (which will be ignored, or NaN). Remove it
    amb_df['chamber_block'] = amb_df['amb_block']
    amb_df = amb_df.loc[~amb_df['amb_block'].isna()]
    amb_df.drop('amb_block', axis=1, inplace=True)
    oc_df['chamber_block'] = oc_df['amb_block']
    oc_df = oc_df.loc[~oc_df['amb_block'].isna()]
    oc_df.drop('amb_block', axis=1, inplace=True)
    # Make sure the dtype is the same as the chamber block
    amb_df['chamber_block'] = amb_df['chamber_block'].astype(int)
    oc_df['chamber_block'] = oc_df['chamber_block'].astype(int)
    
    # Merge with data. Note: Removes the ambient from the temp df before merging, so that no ambient-to-ambient flux is calculated
    merged = temp.loc[temp['status'] != 'amb'].merge(amb_df, on='chamber_block', how='left')
    merged = merged.merge(oc_df, on='chamber_block', how='left')
    
    return(merged)

# Calculate all fluxes for all gases
def calculate_fluxes(temp):
    temp = temp.copy()
    
    # The TSI flow meters use SLPM with the following standard values for air flow:
    air_temperature_tsi_c = 21.11 # [°C]
    air_pressure_tsi_kpa  = 101.3 # [kPa]
    
    # Fluxes with total surface area, between chamber and ambient measurements
    temp['flux.h2o.ch_amb.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    temp['flux.co2.ch_amb.umol_m2_s1'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co2.mean.umol_mol.ambient'] * 10**(-6),
                                                            temp['conc.co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    temp['flux.cos.ch_amb.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.cos.mean.pmol_mol.ambient'] * 10**(-12),
                                                            temp['conc.cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    temp['flux.co.ch_amb.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co.nmol_mol.ambient'] * 10**(-9),
                                                            temp['conc.co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    
    # Fluxes with projected leaf area, between chamber and ambient measurements
    temp['flux.h2o.ch_amb.proj_la.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    temp['flux.co2.ch_amb.proj_la.umol_m2_s1'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co2.mean.umol_mol.ambient'] * 10**(-6),
                                                            temp['conc.co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    temp['flux.cos.ch_amb.proj_la.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.cos.mean.pmol_mol.ambient'] * 10**(-12),
                                                            temp['conc.cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    temp['flux.co.ch_amb.proj_la.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.ambient'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co.nmol_mol.ambient'] * 10**(-9),
                                                            temp['conc.co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    # For soil chambers, there is no projected leaf area
    temp.loc[temp['type'] == 'soil', 'flux.h2o.ch_amb.proj_la.mmol_m2_s'] = np.nan
    temp.loc[temp['type'] == 'soil', 'flux.co2.ch_amb.proj_la.umol_m2_s1'] = np.nan
    temp.loc[temp['type'] == 'soil', 'flux.cos.ch_amb.proj_la.pmol_m2_s'] = np.nan
    temp.loc[temp['type'] == 'soil', 'flux.co.ch_amb.proj_la.nmol_m2_s'] = np.nan
    
    # Fluxes with total surface area, between chamber and open chamber measurements
    temp['flux.h2o.ch_oc.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    temp['flux.co2.ch_oc.umol_m2_s1'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co2.mean.umol_mol.oc'] * 10**(-6),
                                                            temp['conc.co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    temp['flux.cos.ch_oc.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.cos.mean.pmol_mol.oc'] * 10**(-12),
                                                            temp['conc.cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    temp['flux.co.ch_oc.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co.nmol_mol.oc'] * 10**(-9),
                                                            temp['conc.co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2'])
    
    # Fluxes with projected leaf area, between chamber and open chamber measurements
    temp['flux.h2o.ch_oc.proj_la.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    temp['flux.co2.ch_oc.proj_la.umol_m2_s1'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co2.mean.umol_mol.oc'] * 10**(-6),
                                                            temp['conc.co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    temp['flux.cos.ch_oc.proj_la.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.cos.mean.pmol_mol.oc'] * 10**(-12),
                                                            temp['conc.cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    temp['flux.co.ch_oc.proj_la.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['conc.h2o.mmol_mol.oc'],
                                                            temp['conc.h2o.mmol_mol'],
                                                            temp['conc.co.nmol_mol.oc'] * 10**(-9),
                                                            temp['conc.co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['surface.area.m2']/proj_la_factor)
    # For soil chambers, there is no projected leaf area
    temp.loc[temp['type'] == 'soil', 'flux.h2o.ch_oc.proj_la.mmol_m2_s'] = np.nan
    temp.loc[temp['type'] == 'soil', 'flux.co2.ch_oc.proj_la.umol_m2_s1'] = np.nan
    temp.loc[temp['type'] == 'soil', 'flux.cos.ch_oc.proj_la.pmol_m2_s'] = np.nan
    temp.loc[temp['type'] == 'soil', 'flux.co.ch_oc.proj_la.nmol_m2_s'] = np.nan
    
    # There is no flux between open chamber and open chamber
    temp.loc[temp['status'] == 'oc', 'flux.h2o.ch_oc.mmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co2.ch_oc.umol_m2_s1'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.cos.ch_oc.pmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co.ch_oc.nmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.h2o.ch_oc.proj_la.mmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co2.ch_oc.proj_la.umol_m2_s1'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.cos.ch_oc.proj_la.pmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co.ch_oc.proj_la.nmol_m2_s'] = np.nan
    
    return(temp)

# Calculate all fluxes for all gases
def calculate_conductances(temp):
    temp = temp.copy()
    
    # Calculate VPD
    temp['VPD.Pa'] = calculate_VPD(temp['temp.air.current.chamber.c'], temp['conc.h2o.mmol_mol.ambient'], temp['P.irga.kPa']*1000)
    
    # Conductances for COS with total leaf area, between chamber and ambient measurements
    temp['cond.stomatal.ch_amb.cos'] = calculate_cos_stomatal_conductance_ball_berry(temp['temp.air.current.chamber.c'],
                                                                    temp['conc.h2o.mmol_mol.ambient'],
                                                                    temp['P.irga.kPa']*1000,
                                                                    temp['flux.h2o.ch_amb.mmol_m2_s'],
                                                                    temp['flux.co2.ch_amb.umol_m2_s1'],
                                                                    temp['conc.co2.mean.umol_mol.ambient'],
                                                                    temp['par.ambient.umol_m2_s1'])
    temp['cond.total.ch_amb.cos'] = calculate_cos_total_conductance(temp['flux.cos.ch_amb.pmol_m2_s'],
                                                                    temp['conc.cos.mean.pmol_mol.ambient'])
    temp['cond.internal.ch_amb.cos'] = calculate_cos_internal_conductance(temp['cond.total.ch_amb.cos'],
                                                                    temp['cond.stomatal.ch_amb.cos'])
    temp['rel_uptake.ch_amb'] = relative_uptake(temp['flux.cos.ch_amb.pmol_m2_s'],
                                                                    temp['flux.co2.ch_amb.umol_m2_s1'],
                                                                    temp['conc.cos.mean.pmol_mol.ambient'],
                                                                    temp['conc.co2.mean.umol_mol.ambient'])
    
    # Conductances for COS with projected leaf area, between chamber and ambient measurements
    temp['cond.stomatal.ch_amb.proj_la.cos'] = calculate_cos_stomatal_conductance_ball_berry(temp['temp.air.current.chamber.c'],
                                                                    temp['conc.h2o.mmol_mol.ambient'],
                                                                    temp['P.irga.kPa']*1000,
                                                                    temp['flux.h2o.ch_amb.proj_la.mmol_m2_s'],
                                                                    temp['flux.co2.ch_amb.proj_la.umol_m2_s1'],
                                                                    temp['conc.co2.mean.umol_mol.ambient'],
                                                                    temp['par.ambient.umol_m2_s1'])
    temp['cond.total.ch_amb.proj_la.cos'] = calculate_cos_total_conductance(temp['flux.cos.ch_amb.proj_la.pmol_m2_s'],
                                                                    temp['conc.cos.mean.pmol_mol.ambient'])
    temp['cond.internal.ch_amb.proj_la.cos'] = calculate_cos_internal_conductance(temp['cond.total.ch_amb.proj_la.cos'],
                                                                    temp['cond.stomatal.ch_amb.proj_la.cos'])
    temp['rel_uptake.ch_amb.proj_la'] = relative_uptake(temp['flux.cos.ch_amb.proj_la.pmol_m2_s'],
                                                                    temp['flux.co2.ch_amb.proj_la.umol_m2_s1'],
                                                                    temp['conc.cos.mean.pmol_mol.ambient'],
                                                                    temp['conc.co2.mean.umol_mol.ambient'])
    
    # Conductances for COS with total leaf area, between chamber and open chamber measurements
    temp['cond.stomatal.ch_oc.cos'] = calculate_cos_stomatal_conductance_ball_berry(temp['temp.air.current.chamber.c'],
                                                                    temp['conc.h2o.mmol_mol.oc'],
                                                                    temp['P.irga.kPa']*1000,
                                                                    temp['flux.h2o.ch_oc.mmol_m2_s'],
                                                                    temp['flux.co2.ch_oc.umol_m2_s1'],
                                                                    temp['conc.co2.mean.umol_mol.oc'],
                                                                    temp['par.ambient.umol_m2_s1'])
    temp['cond.total.ch_oc.cos'] = calculate_cos_total_conductance(temp['flux.cos.ch_oc.pmol_m2_s'],
                                                                    temp['conc.cos.mean.pmol_mol.oc'])
    temp['cond.internal.ch_oc.cos'] = calculate_cos_internal_conductance(temp['cond.total.ch_oc.cos'],
                                                                    temp['cond.stomatal.ch_oc.cos'])
    temp['rel_uptake.ch_oc'] = relative_uptake(temp['flux.cos.ch_oc.pmol_m2_s'],
                                                                    temp['flux.co2.ch_oc.umol_m2_s1'],
                                                                    temp['conc.cos.mean.pmol_mol.oc'],
                                                                    temp['conc.co2.mean.umol_mol.oc'])
    
    # Conductances for COS with projected leaf area, between chamber and open chamber measurements
    temp['cond.stomatal.ch_oc.proj_la.cos'] = calculate_cos_stomatal_conductance_ball_berry(temp['temp.air.current.chamber.c'],
                                                                    temp['conc.h2o.mmol_mol.oc'],
                                                                    temp['P.irga.kPa']*1000,
                                                                    temp['flux.h2o.ch_oc.proj_la.mmol_m2_s'],
                                                                    temp['flux.co2.ch_oc.proj_la.umol_m2_s1'],
                                                                    temp['conc.co2.mean.umol_mol.oc'],
                                                                    temp['par.ambient.umol_m2_s1'])
    temp['cond.total.ch_oc.proj_la.cos'] = calculate_cos_total_conductance(temp['flux.cos.ch_oc.proj_la.pmol_m2_s'],
                                                                    temp['conc.cos.mean.pmol_mol.oc'])
    temp['cond.internal.ch_oc.proj_la.cos'] = calculate_cos_internal_conductance(temp['cond.total.ch_oc.proj_la.cos'],
                                                                    temp['cond.stomatal.ch_oc.proj_la.cos'])
    temp['rel_uptake.ch_oc.proj_la'] = relative_uptake(temp['flux.cos.ch_oc.proj_la.pmol_m2_s'],
                                                                    temp['flux.co2.ch_oc.proj_la.umol_m2_s1'],
                                                                    temp['conc.cos.mean.pmol_mol.oc'],
                                                                    temp['conc.co2.mean.umol_mol.oc'])
    
    # Biochemical conductance
    temp['cond.biochemical'] = np.nan
    temp.loc[temp['plot'] == 'irrigation', 'cond.biochemical'] = calculate_biochemical_conductance(temp['temp.leaf.current.chamber.c'], LAI_irr)
    temp.loc[temp['plot'] == 'control',    'cond.biochemical'] = calculate_biochemical_conductance(temp['temp.leaf.current.chamber.c'], LAI_ctr)
    
    # Mesophyll conductance
    temp['cond.mesophyll'] = np.nan
    temp.loc[temp['plot'] == 'irrigation', 'cond.mesophyll'] = calculate_mesophyll_conductance(temp['temp.leaf.current.chamber.c'], LAI_irr)
    temp.loc[temp['plot'] == 'control',    'cond.mesophyll'] = calculate_mesophyll_conductance(temp['temp.leaf.current.chamber.c'], LAI_ctr)

    return(temp)

def calibrate_co2_h2o(temp, cal_stats):
    temp = temp.copy()
    
    # Correlation of H2O
    print('    Correlation between IRGA ambient and laser')
    h2o_mask = (temp['status'] == 'amb') & (temp['conc.h2o.irga.ambient.mmol_mol'].notna()) & (temp['conc.h2o.mmol_mol'].notna())
    n_h2o = len(temp.loc[h2o_mask,'conc.h2o.mmol_mol'])
    if(n_h2o >= 10):
        h2o_corr = stats.linregress(temp.loc[h2o_mask,'conc.h2o.mmol_mol'], temp.loc[h2o_mask,'conc.h2o.irga.ambient.mmol_mol'])
        print('        H2O: ', h2o_corr, n_h2o)
    # Correlation of CO2
    co2_mask = (temp['status'] == 'amb') & (temp['conc.co2.irga.ambient.umol_mol'].notna()) & (temp['conc.co2.mean.umol_mol'].notna())
    n_co2 = len(temp.loc[co2_mask,'conc.co2.mean.umol_mol'])
    if(n_co2 >= 10):
        co2_corr = stats.linregress(temp.loc[co2_mask,'conc.co2.mean.umol_mol'], temp.loc[co2_mask,'conc.co2.irga.ambient.umol_mol'])
        print('        CO2: ', co2_corr, n_co2)
    
    # Flag the data:
    # H2O bad  = 1
    # CO2 bad  = 2
    # Both bad = 3
    temp['flag'] = np.nan
    if((n_h2o < 10) | (n_co2 < 10)):
        temp['flag'] = 0
    else:
        if(h2o_corr.rvalue < 0.90):
            temp['flag'] = 1
        if(co2_corr.rvalue < 0.90):
            temp['flag'] = 2
        if((h2o_corr.rvalue < 0.90) & (co2_corr.rvalue < 0.90)):
            temp['flag'] = 3
        
    # Add information to the calibration statistics df
    temp_cal_df = pd.DataFrame(columns=['date','conc.h2o.slope','conc.h2o.intercept','conc.h2o.r2','conc.h2o.p','conc.h2o.sd','conc.h2o.n',\
                                        'conc.co2.slope','conc.co2.intercept','conc.co2.r2','conc.co2.p','conc.co2.sd','conc.co2.n'], index=[0])
    temp_cal_df['date'] = temp['timestamp'].dt.date.unique()
    if(n_h2o >= 10):
        temp_cal_df['conc.h2o.slope']     = h2o_corr.slope
        temp_cal_df['conc.h2o.intercept'] = h2o_corr.intercept
        temp_cal_df['conc.h2o.r2']        = h2o_corr.rvalue
        temp_cal_df['conc.h2o.p']         = h2o_corr.pvalue
        temp_cal_df['conc.h2o.sd']        = h2o_corr.stderr
    temp_cal_df['conc.h2o.n'] = n_h2o
    if(n_co2 >= 10):
        temp_cal_df['conc.co2.slope']     = co2_corr.slope
        temp_cal_df['conc.co2.intercept'] = co2_corr.intercept
        temp_cal_df['conc.co2.r2']        = co2_corr.rvalue
        temp_cal_df['conc.co2.p']         = co2_corr.pvalue
        temp_cal_df['conc.co2.sd']        = co2_corr.stderr
    temp_cal_df['conc.co2.n'] = n_co2
    cal_stats = pd.concat([cal_stats, temp_cal_df])
        
    # Apply corrections
    temp['conc.h2o.calibrated.mmol_mol'] = np.nan
    temp['conc.co2.calibrated.umol_mol'] = np.nan
    if(n_h2o >= 10):
        if(h2o_corr.rvalue > 0.75):
            temp['conc.h2o.calibrated.mmol_mol'] = h2o_corr.slope * temp['conc.h2o.mmol_mol'] + h2o_corr.intercept
    if(n_co2 >= 10):
        if(co2_corr.rvalue > 0.50):
            temp['conc.co2.calibrated.umol_mol'] = co2_corr.slope * temp['conc.co2.mean.umol_mol'] + co2_corr.intercept
    
    return(temp, cal_stats)

# Writes output files averaged by chamber and time
def write_stats_file(out_df, out_dir):
    # Check if output folders exist. If not, create
    if(not os.path.exists(out_dir)):
        os.makedirs(out_dir)
    # Create file name
    out_fn = out_dir + '/calibration_stats_co2_h2o.csv'
    # Remove timezone information
    #out_df['timestamp'] = out_df['timestamp'].dt.tz_localize(None)
    # If the file does not exist, create it with the header
    out_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d')

# Writes output files averaged by chamber and time
def write_summarised_file(out_df, date_idx, out_dir):
    # Check if output folders exist. If not, create
    this_month = str(date_idx)[0:6]
    if(not os.path.exists(out_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir)
    # Create file name
    out_fn = out_dir + '/' + output_fn + '_' + str(this_month) + '.csv'
    # Remove timezone information
    #out_df['timestamp'] = out_df['timestamp'].dt.tz_localize(None)
    # If the monthly file does not exist, create it with the header
    if not os.path.isfile(out_fn):
        out_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')
    else: # otherwise append without writing the header
        out_df.to_csv(out_fn, mode='a', header=False, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')

# Full processing pipeline

In [ ]:
# Applies all the corrections in order. This gets called by the main loops that are in control of reading and writing files
def apply_corrections(laser, amb, par, tc, flow, cal_stats, date_idx):
    # Show info
    print('- Applying corrections on ', np.unique(laser['dayid'])[0], '...')
    # Add average columns of the different gases, and standard deviation
    laser = fix_gas_units(laser)
    # get the right PAR, flow and temperature data
    par_df  = get_relevant_data(par, par_log, date_idx, name='par', unit='umol_m2_s1', old_name='par_Avg(', calibration=100)
    tc_df   = get_relevant_data(tc, tc_log, date_idx, name='temp', unit='c', old_name='Tc(')
    flow_df = get_relevant_data(flow, fm_log, date_idx, name='pump.flow', unit='slpm', old_name='FlowMeter(')
    # Merge all data
    temp = add_ambient(laser, amb)
    temp = add_sensor_data(temp, par_df, interpolation_limit=120) # Interpolation for max. 2min (=120s)
    temp = add_sensor_data(temp, tc_df, interpolation_limit=120)    # Interpolation for max. 4 values (=4s)
    temp = add_sensor_data(temp, flow_df, interpolation_limit=120)  # Interpolation for max. 4 values (=4s)
    temp = fix_PAR(temp) # Fix nighttime PAR data
    # Temporarily add counter and chamber
    temp = make_counter_and_chamber(temp, date_idx)
    # Define day/night
    temp = define_day_night(temp)
    # Checks if there is enough data left to properly do all the calculations
    if(len(temp['timestamp']) > 240):
        # Add a column for current PAR, temperature and current flow
        temp = add_current_variable(temp)
        # Add leaf area
        temp = add_surface_area(temp, date_idx, sa_log)
        # Add the pump flow rate from the log file, if not previously given by measurements
        temp = add_log_flow(temp)
        # Remove bad chambers
        temp = remove_bad_chambers(temp)
        # Remove the data outside the averaging period, if specified
        temp = remove_obsolete_data(temp, averaging_period)
        # Daily correction of CO2 and H2O by correlating to IRGA values, if available
        temp, cal_stats = calibrate_co2_h2o(temp, cal_stats)
    else:
        print('- Data of', date_idx, 'too short for corrections')
        
    return(temp, cal_stats)

def calculate_values(temp, date_idx):
    # Make means, standard deviations, and
    # remove all except last number of seconds, provided in the variable 'averaging_period'
    temp = make_means(temp, averaging_period)
    # Find the corresponding ambient measurements
    if(find_ambient == 'interpolated'):
        temp = find_corresponding_ambient_interpolated(temp)
    elif(find_ambient == 'next'):
        temp = find_corresponding_ambient_next(temp)
    else:
        raise ValueError('Invalid choice: Please choose which method is used to find ambient data')
    # calculate fluxes
    temp = calculate_fluxes(temp)
    # Calculate conductances
    temp = calculate_conductances(temp)
    
    return(temp)

In [ ]:
# Run calculations
#-----------------

# List all files in the directories
laser_fn_list = sorted(glob.glob(project_path_laser + '*/*', recursive=True))
amb_fn_list   = sorted(glob.glob(project_path_amb + '*/*', recursive=True))
par_fn_list   = sorted(glob.glob(project_path_par + '*/*', recursive=True))
tc_fn_list    = sorted(glob.glob(project_path_tc + '*/*', recursive=True))
flow_fn_list  = sorted(glob.glob(project_path_flow + '*/*', recursive=True))
saved = []

# Prepare 1h directory. Because files aren't overwritten typically, we need to empty it first
empty_dir(project_path_output + find_ambient + '/' + '1h/')
empty_dir(project_path_output + find_ambient + '/' + '1min/')
empty_dir(project_path_output + find_ambient + '/' + 'stats_calibration/')

# Prepare empty dataframe for calibration statistics
cal_stats_df = pd.DataFrame(columns=['date','conc.h2o.slope','conc.h2o.intercept','conc.h2o.r2','conc.h2o.p','conc.h2o.sd','conc.h2o.n',\
                                     'conc.co2.slope','conc.co2.intercept','conc.co2.r2','conc.co2.p','conc.co2.sd','conc.co2.n'])

for fn_i, fn in enumerate(laser_fn_list):
    # Only run data in the month list
    current_month = fn.replace(project_path_laser[:-1], '')[1:8]
    if((current_month in month_list) or (len(month_list) == 0)):
        pass
    else:
        continue
    
    # Determine current day
    this_day = fn[-12:-4]
    
    # Debugging message
    if(fn_i % 1 == 0): # % 20 to show every 20th file being loaded
        print( '{:<07}'.format(str(round(fn_i * 100 / len(laser_fn_list), 4))) + '%\t' + str(this_day[0:4]) + '-' + str(this_day[4:6]) + '-' + str(this_day[6:8])) # Show status
        
    # Now find corresponding files and load all the data
    laser_df = read_data_file(fn)
    amb_df  = find_and_load_file(amb_fn_list, this_day)
    par_df  = find_and_load_file(par_fn_list, this_day)
    tc_df   = find_and_load_file(tc_fn_list, this_day)
    flow_df = find_and_load_file(flow_fn_list, this_day)
    
    # Now apply corrections
    out_df, cal_stats_df = apply_corrections(laser_df, amb_df, par_df, tc_df, flow_df, cal_stats_df, this_day)
    
    # If there is any data left after corrections, i.e. it wasn't all deleted due to bad data
    if(len(out_df) > 0):
        # Write output in full temporal resolution
        write_output_file(out_df, this_day, project_path_output + find_ambient + '/' + 'raw_qc/')
        
        # Create means and calculate fluxes for the averaging period
        out_df = calculate_values(out_df, this_day)
        
        # Write output averaged output file
        write_summarised_file(out_df, this_day, project_path_output + find_ambient + '/' + '1min/')
    pass

write_stats_file(cal_stats_df, project_path_output + find_ambient + '/' + 'stats_calibration/')
print('Done...')

## Diagnostics graphs

In [ ]:
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
# Colours
cbPalette = ['#000000', '#E69F00', '#0072B2', '#CC00CC', '#009E73', '#D55E00', '#CC79A7', '#FF3300', '#F0E442', '#56B4E9']

plt = ggplot(out_df)
plt = plt + geom_point(aes(x='conc.h2o.mmol_mol.ambient', y='conc.h2o.irga.ambient.mmol_mol', colour='plot'), size=0.5)
plt = plt + labs(x='Laser $H_2 O$', y='IRGA $H_2 O$', colour='Type', parse=True)
plt = plt + scale_colour_manual(values=cbPalette)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family='serif'), axis_text_x=element_text(rotation=90, hjust=0.5))
print(plt)

plt = ggplot(out_df)
plt = plt + geom_point(aes(x='conc.co2.mean.umol_mol.ambient', y='conc.co2.irga.ambient.umol_mol', colour='plot'), size=0.5)
plt = plt + labs(x='Laser $CO_2$', y='IRGA $CO_2$', colour='Type', parse=True)
plt = plt + scale_colour_manual(values=cbPalette)
plt = plt + theme_bw()
plt = plt + theme(text=element_text(family='serif'), axis_text_x=element_text(rotation=90, hjust=0.5))
print(plt)